# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
!pip install pandas

In [9]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [10]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
# Replace None by your own value and justify
llm = ChatOpenAI(temperature=1.3)

# Middle ground between deterministic and random with a bit more creativity

In [55]:
prompt = ChatPromptTemplate.from_template(
    "Describe the features and benefits of {product}."
)

In [56]:

chain = LLMChain(llm=llm, prompt=prompt)

In [57]:
product = "Luxury Air Mattress"
chain.run(product)

"A luxury air mattress typically boasts several features that set it apart from traditional air mattresses. Some of these features include:\n\n1. High-quality materials: Luxury air mattresses are often made with durable and luxurious materials such as memory foam, PVC, or other high-grade materials that provide better support and comfort.\n\n2. Customizable firmness: Many luxury air mattresses come with adjustable firmness settings, allowing users to customize the level of support to their liking.\n\n3. Built-in air pumps: Luxury air mattresses often come with built-in air pumps that make inflating and deflating the mattress quick and easy.\n\n4. Enhanced support: Luxury air mattresses often feature additional support layers, such as reinforced edges or added lumbar support, to provide a more comfortable and restful sleep experience.\n\n5. Temperature regulation: Some luxury air mattresses come with temperature-regulating features, such as cooling gel layers, to help keep you comfortab

## SimpleSequentialChain

In [58]:
from langchain.chains import SimpleSequentialChain

In [59]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Generate a creative name for a {product}."
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [60]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a catchy slogan for the product named {product_name}."
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [61]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [62]:
overall_simple_chain.run("Waterproof Phone Pouch")
overall_simple_chain.run("Pillows Insert")



> Entering new SimpleSequentialChain chain...
AquaGuard Phone Seal
Seal out the water, seal in the protection with AquaGuard Phone Seal!

> Finished chain.


> Entering new SimpleSequentialChain chain...
Fluff-Flow™ Pillow Insert
"Sweet dreams start with Fluff-Flow™ - the perfect pillow insert!"

> Finished chain.


'"Sweet dreams start with Fluff-Flow™ - the perfect pillow insert!"'

**Repeat the above twice for different products**

## SequentialChain

In [63]:
from langchain.chains import SequentialChain

In [64]:
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to French: {Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="French_Review"
                    )

In [65]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in one sentence: {Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Summary"
                    )


In [66]:
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to English: {French_Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="English_Review"
                      )

In [67]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the review summary '{Summary}', suggest a follow-up question for the reviewer."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Follow_Up"
                     )


In [68]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["French_Review", "Summary", "English_Review", "Follow_Up"],
    verbose=True
)

In [69]:
review = df.Review[5]
overall_chain(review)

C:\Users\jurge\AppData\Local\Temp\ipykernel_1372\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'French_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Summary': "Le reviewer a trouvé le goût médiocre, la mousse ne tient pas et se demande s'il s'agit d'un vieux lot ou d'une contrefaçon.",
 'English_Review': "I find the taste average. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Follow_Up': 'Quels éléments vous ont amené à penser que ce lot pourrait être vieux ou une contrefaçon?'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [70]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [71]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [72]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [73]:
llm = ChatOpenAI(temperature=0)

In [74]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [75]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [76]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [77]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [78]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\jurge\AppData\Local\Temp\ipykernel_1372\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [79]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [80]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [83]:
chain.run("How does a route chain fit together?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'How does a route chain fit together?'}
> Finished chain.


'In computer science, a route chain refers to a sequence of interconnected routes or paths that data or information can travel through in a network or system. Each route in the chain is connected to the next, forming a continuous path for data to flow.\n\nTo understand how a route chain fits together, we can think of it as a series of interconnected nodes or points, where each node represents a specific route or path. The data or information travels from one node to the next, following the defined path until it reaches its destination.\n\nIn terms of implementation, a route chain can be represented using data structures such as linked lists, arrays, or graphs. Each route in the chain is represented as a node in the data structure, with pointers or references to the next route in the sequence.\n\nWhen designing a route chain, it is important to consider factors such as efficiency, scalability, and reliability. By carefully planning and organizing the routes in the chain, we can ensure t

**Repeat the above at least once for different inputs and chains executions - Be creative!**